# Расчётное задание по системам массового обслуживания
## 1. Определить вероятность блокировки пакета днем и ночью, если входной и выходной поток являются пуассоновскими
Используем формулу для подсчета вероятности отказа многоканальной СМО с очередью
нам даны: количество концентраторов, интенсивность днём и ночью, 
максимальный размер очереди (размер буфера минус 1) 
и выходная интенсивность (скорость передачи, деленная на среднюю длину пакета 2400/1200)

In [1]:
import numpy as np
import random

# Количество концентраторов
k = 2
# Интесивность входного потока
lambda_day = 2
# Интенсивность ночного потока
lambda_night = 0.5
# Максимальный размер очереди
m = 6
# Интенсивность выходного потока
mu = 2
# Количество обслуженных заявок в единицу времени
p_day = lambda_day / mu
p_night = lambda_night / mu
# Предельная вероятность
p0 = 0
f = 1
for i in range(1, k + 1):
    f *= i
    p0 += (p_day ** i) / f
p0 = (p0 + 1 + (((p_day ** (k + 1)) / (k * f)) * (1 - (p_day / k) ** m) / (1 - p_day / k))) ** (-1)
# Вероятность отказа
pd = (p_day ** (k + m) / ((k ** m) * f)) * p0

p0 = 0
f = 1
for i in range(1, k + 1):
    f *= i
    p0 += (p_night ** i) / f
p0 = (p0 + 1 + (((p_night ** (k + 1)) / (k * f)) * (1 - (p_night / k) ** m) / (1 - p_night / k))) ** (-1)
# Вероятность отказа
pe = (p_night ** (k + m) / ((k ** m) * f)) * p0

print('Вероятность отказа днём:', pd)
print('Вероятность отказа ночью:', pe)



Вероятность отказа днём: 0.0026109660574412533
Вероятность отказа ночью: 9.271833754537287e-08


## 2. Определить вероятность блокировки пакета в задаче 1, используя метод Монте-Карло. Сравнить результаты
Исходные данные те же, количество пакетов рассчитываем так: по условию день - 16 часов, ночь - 8 часов
мы переводим это в секунды и умножаем на соответсвующую интенсивность. 
Повторы берем так, чтобы слишком долго не считало =), но чем больше повторов - тем больше точность


In [3]:
import numpy as np
import random


def alg(lambda_day, n_packets):
    # Количество концентраторов
    k = 2
    # Инициализация генеротора случайных чисел
    np.random.seed(100)
    # Вероятность блокировки
    p_refuse = 0
    # Количество повторов в методе Монте-Карло
    n_rep = 50
    # Имитационное моделирование
    # Интенсивность выходного потока
    mu = 2
    for i in range(n_rep):

        # Текущее время
        t = 0
        # текущий размер очереди
        queue = 0
        # Максимальный размер очереди
        m = 6
        # время, когда прибор освободится
        t_free = []
        for j in range(k):
            t_free.append(0)
        # Количество потярянных пактеов
        n_lost = 0
        for f in range(n_packets):
            # время поступления нового пакета
            t += np.random.exponential(scale=1 / lambda_day)
            # проверяем, свободен ли концентратор
            n = 0
            if queue > 0:
                for j in range(k):
                    while t_free[j] < t and queue > 0:
                        t_free[j] += np.random.exponential(scale=1 / mu)
                        queue -= 1
            for h in range(k):
                if t_free[h] < t:
                    t_free[h] = t + np.random.exponential(scale=1 / mu)
                    break
                else:
                    n += 1
            if n == k:
                if queue < m:
                    queue += 1
                else:
                    n_lost += 1
        p_refuse += n_lost / n_packets
    return p_refuse


# Оценка вероятности блокировки
# Количество повторов в методе Монте-Карло
n_rep = 50
# Количество пакетов (3600 секунд в часе, интенсивность)
p_refuse1 = alg(2, 16 * 3600 * 2) / n_rep
p_refuse2 = alg(0.5, 4 * 3600) / n_rep
print('Вероятность отказа днём: ' + str(p_refuse1))
print('Вероятность отказа ночью: ' + str(p_refuse2))

Вероятность отказа днём: 0.002669791666666666
Вероятность отказа ночью: 0.0


Вероятность отказа днём сходится, а ночью метод Монте-Карло выдаёт нулевую, потому что проверок слишком мало, она стремится к нулю, надо хотя бы ~миллион проверок

## 3. Определить оптимальный размер буфера, чтобы вероятность блокировки пакета не превышала 0.0001
В третьем пункте считается по циклу наращивая очередь (это буфер декрементированный на единицу, т.е. B - 1) пока вероятность не станет меньше 0.0001
Можно делать по первому пункту, можно по второму, сделал через Монте-Карло

In [5]:
import numpy as np

# Инициализация генеротора случайных чисел
np.random.seed(100)


def calculations(k, lambda_, mu):
    # Количество повторов в методе Монте-Карло
    n_rep = 100
    # Вероятность блокировки
    p_refuse = 0
    # Имитационное моделирование
    m = 0
    while 1:
        for i in range(n_rep):
            # текущий размер очереди
            queue = 0
            # Количество пакетов
            n_packets = 1000
            # Текущее время
            t = 0
            # время, когда прибор освободится

            t_free = []
            for j in range(k):
                t_free.append(0)
            # Количество потярянных пактеов
            n_lost = 0
            for f in range(n_packets):
                # время поступления нового пакета
                t += np.random.exponential(scale=1 / lambda_)
                # проверяем, свободен ли концентратор
                n = 0
                if queue > 0:
                    for j in range(k):
                        while t_free[j] < t and queue > 0:
                            t_free[j] += np.random.exponential(scale=1 / mu)
                            queue -= 1
                for h in range(k):
                    if t_free[h] < t:
                        t_free[h] = t + np.random.exponential(scale=1 / mu)
                        break
                    else:
                        n += 1
                if n == k:
                    if queue < m:
                        queue += 1
                    else:
                        n_lost += 1
            p_refuse += n_lost / n_packets
        p_refuse /= n_rep
        if p_refuse < 0.0001:
            break
        else:
            m += 1
            p_refuse = 0

    # Оценка вероятности блокировки
    return p_refuse, m + 1


# Количество концентраторов
k_ = 2

# Интенсивность потока
mu_ = 2400 / 1200

lambda_day = 2
lambda_night = 0.5
p_day, b_day = calculations(k_, lambda_day, mu_)
p_night, b_night = calculations(k_, lambda_night, mu_)

print('Вероятность отказа днём: ' + str(p_day) + ' < 0.0001')
print('Размер буфера днём: ' + str(b_day))
print('Вероятность отказа ночью: ' + str(p_night) + ' < 0.0001')
print('Размер буфера ночью: ' + str(b_night))

Вероятность отказа днём: 6e-05 < 0.0001
Размер буфера днём: 12
Вероятность отказа ночью: 1e-05 < 0.0001
Размер буфера ночью: 4


Получается, что днём надо взять буфер 12 (очередь = 11), а ночью 4 (очередь = 3)